In [1]:
!pip install transformers datasets accelerate bitsandbytes peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.3 MB/s eta 0:00:00


In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig

)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from google.colab import files

model_name = "google-t5/t5-small"

# ----------------------------
# 1. Quantization QLoRA (4-bit)
# ----------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,               # quantization 4-bit
    bnb_4bit_use_double_quant=True,  # double quantization (optimise RAM)
    bnb_4bit_quant_type="nf4",       # NF4 = meilleure précision
    bnb_4bit_compute_dtype="bfloat16"
)

# ----------------------------
# 2. Charger modèle en 4-bit
# ----------------------------
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# ----------------------------
# 3. Activer LoRA (adapters)
# R = 16 est standard
# Chaque couche gelée sauf matrices Q et V
# ----------------------------
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q","v","k","o","wi","wo"],
    #target_modules=["q", "v",],  # T5 layers qui supportent LoRA
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)

print(model)  # (Optionnel) tu vois que seules q,v sont entraînables

# ----------------------------
# 4. Préparer dataset
# ----------------------------
#dataset = load_dataset("json", data_files="/kaggle/input/content/nmap_dataset.json")
uploaded = files.upload()  # Sélectionne 'nmap_dataset_prof.json' depuis ton PC

# ------------------------------
# 2. Charger dataset
# ------------------------------
dataset_path = "/content/nmap_dataset.json"
dataset = load_dataset(
    "json",
    data_files=dataset_path,
    split="train"
)

print("Dataset chargé :", dataset[0])

def encode(example):
    # Input
    inputs = tokenizer(
        example["input"],
        truncation=True,
        padding="max_length",
        max_length=128  # ← Réduit de 256 à 128
    )

    # Output
    with tokenizer.as_target_tokenizer():
        outputs = tokenizer(
            example["output"],
            truncation=True,
            padding="max_length",
            max_length=64  # ← Réduit de 256 à 64
        )

    inputs["labels"] = outputs["input_ids"]

    # CRITIQUE : Remplace padding par -100 (ignoré dans la loss)
    inputs["labels"] = [
        (label if label != tokenizer.pad_token_id else -100)
        for label in inputs["labels"]
    ]

    return inputs

dataset = dataset.map(encode)

# ----------------------------
# 5. Data collator
# ----------------------------
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# ----------------------------
# 6. Training arguments
# ----------------------------
training_args = TrainingArguments(
    output_dir="./t5-nmap-qlora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,            # conseillé pour QLoRA
    num_train_epochs=15,
    fp16=False,
    bf16=True,                     # T5 + QLoRA = BF16
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
)

# ----------------------------
# 7. Trainer
# ----------------------------

splits = dataset.train_test_split(test_size=0.1)
train_dataset = splits["train"]
test_dataset  = splits["test"]

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
# ----------------------------
# 8. Fine-tuning
# ----------------------------
trainer.train()



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss
50,4.308700
100,2.755700
150,2.004500
200,1.681600
250,1.513300
300,1.329100
350,1.210500
400,1.161000
450,1.085000
500,1.029000


TrainOutput(global_step=1530, training_loss=1.1228978861391155, metrics={'train_runtime': 1084.8795, 'train_samples_per_second': 22.385, 'train_steps_per_second': 1.41, 'total_flos': 862029946552320.0, 'train_loss': 1.1228978861391155, 'epoch': 15.0})

In [7]:
# ----------------------------
# TEST IMMÉDIAT
# ----------------------------
print("\n" + "="*60)
print("🧪 TEST IMMÉDIAT APRÈS FINE-TUNING")
print("="*60 + "\n")

import torch

model.eval()

test_prompts = [
    "Scan all ports Run default,vuln scripts on 192.168.0.0/16",
    "Scan all ports on 192.168.1.20",
    "Do a ping scan and traceroute on 10.0.0.0/28",
    "Scan top ports with version detection and OS detection on 192.168.1.50",
    "Scan all TCP ports on the target host",
    "Launch the SMB brute-force script using a specific user list and disabling the default password attempts"
]

for prompt in test_prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            num_beams=4,
            do_sample=False
        )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"📝 Prompt : {prompt}")
    print(f"✅ Résultat : {result}\n")


🧪 TEST IMMÉDIAT APRÈS FINE-TUNING

📝 Prompt : Scan all ports Run default,vuln scripts on 192.168.0.0/16
✅ Résultat : nmap -p- --script default,vuln 192.168.0.0/16

📝 Prompt : Scan all ports on 192.168.1.20
✅ Résultat : nmap -p- 192.168.1.20

📝 Prompt : Do a ping scan and traceroute on 10.0.0.0/28
✅ Résultat : nmap -sn --traceroute 10.0.0.0/28

📝 Prompt : Scan top ports with version detection and OS detection on 192.168.1.50
✅ Résultat : nmap -p- -sV -O 192.168.1.50

📝 Prompt : Scan all TCP ports on the target host
✅ Résultat : nmap -p- -O target

📝 Prompt : Launch the SMB brute-force script using a specific user list and disabling the default password attempts
✅ Résultat : nmap --script smb-brute --script-args user-list --script-args target



In [8]:
# ----------------------------
# 9. Sauvegarde modèle LoRA
# ----------------------------

output_dir = "/content/T5-qlora-nmap"

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
import shutil


# -------- ZIP pour téléchargement --------
shutil.make_archive(output_dir, "zip", output_dir)
print("✅ Fine-tuning T5-small avec QLoRA terminé ! Modèle sauvegardé dans :", output_dir)

✅ Fine-tuning T5-small avec QLoRA terminé ! Modèle sauvegardé dans : /content/T5-qlora-nmap
